In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql

import os
from dotenv import load_dotenv

In [8]:
load_dotenv()

# DB credentials
DB_USER = "root"
DB_PASSWORD = ""
DB_HOST = "localhost"
DB_NAME = "airline_analytics"

engine = create_engine(
    f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}"
)

In [3]:
# Define Columns to Read

USE_COLS = [
    "Year", "Month", "DayofMonth", "DayOfWeek",
    "UniqueCarrier", "FlightNum",
    "Origin", "Dest",
    "DepDelay", "ArrDelay",
    "Cancelled", "Diverted",
    "Distance"
]

In [4]:
# Chunk-Based ETL Loop

chunk_size = 50_000
chunk_number = 0

for chunk in pd.read_csv(
    "data/raw/airline.csv",
    encoding="latin1",
    engine="python",
    usecols=USE_COLS,
    chunksize=chunk_size
):
    chunk_number += 1
    print(f"Processing chunk {chunk_number}")

    # --- CLEANING & TRANSFORMATION ---

    # Standardize column names
    chunk.columns = chunk.columns.str.lower()

    # Fill delay NaNs with 0
    chunk["depdelay"] = chunk["depdelay"].fillna(0)
    chunk["arrdelay"] = chunk["arrdelay"].fillna(0)

    # Convert flags to int
    chunk["cancelled"] = chunk["cancelled"].astype(int)
    chunk["diverted"] = chunk["diverted"].astype(int)

    # Create flight_date
    chunk["flight_date"] = pd.to_datetime(
        dict(
            year=chunk["year"],
            month=chunk["month"],
            day=chunk["dayofmonth"]
        )
    )

    # Clean codes
    chunk["uniquecarrier"] = chunk["uniquecarrier"].str.upper().str.strip()
    chunk["origin"] = chunk["origin"].str.upper().str.strip()
    chunk["dest"] = chunk["dest"].str.upper().str.strip()

    # Remove invalid rows
    chunk = chunk[chunk["distance"] > 0]

    # Select columns in staging order
    chunk = chunk[
        [
            "flight_date",
            "year", "month", "dayofmonth", "dayofweek",
            "uniquecarrier", "flightnum",
            "origin", "dest",
            "distance",
            "depdelay", "arrdelay",
            "cancelled", "diverted"
        ]
    ]

    # Rename to match SQL table
    chunk.rename(
        columns={
            "uniquecarrier": "airline_code",
            "flightnum": "flight_number",
            "depdelay": "dep_delay",
            "arrdelay": "arr_delay"
        },
        inplace=True
    )

    # --- INSERT INTO MYSQL ---
    chunk.to_sql(
        "staging_flights",
        engine,
        if_exists="append",
        index=False
    )

print("Data successfully loaded into staging_flights")

Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Processing chunk 7
Processing chunk 8
Processing chunk 9
Processing chunk 10
Processing chunk 11
Processing chunk 12
Processing chunk 13
Processing chunk 14
Processing chunk 15
Processing chunk 16
Processing chunk 17
Processing chunk 18
Processing chunk 19
Processing chunk 20
Processing chunk 21
Processing chunk 22
Processing chunk 23
Processing chunk 24
Processing chunk 25
Processing chunk 26
Processing chunk 27
Processing chunk 28
Processing chunk 29
Processing chunk 30
Processing chunk 31
Processing chunk 32
Processing chunk 33
Processing chunk 34
Processing chunk 35
Processing chunk 36
Processing chunk 37
Processing chunk 38
Processing chunk 39
Processing chunk 40
Processing chunk 41
Processing chunk 42
Processing chunk 43
Processing chunk 44
Processing chunk 45
Processing chunk 46
Processing chunk 47
Processing chunk 48
Processing chunk 49
Processing chunk 50
Processin

#### Load airlines from carriers.csv

In [12]:
carriers = pd.read_csv("data/raw/carriers.csv")

carriers.columns = carriers.columns.str.lower()

carriers = carriers.rename(columns={
    "code": "airline_code",
    "description": "airline_name"
})

carriers = carriers.dropna(subset=["airline_code"])

carriers.to_sql(
    "airlines_dim",
    engine,
    if_exists="append",
    index=False
)

print("airlines_dim loaded")

airlines_dim loaded


In [7]:
airline_sample_df = pd.read_csv(
    "data/raw/airline.csv",
    encoding="latin1",
    engine="python",
    nrows=100_000
)

In [9]:
airline_sample_df.columns = (
    airline_sample_df.columns
      .str.strip()
      .str.lower()
)

In [ ]:
# Load airports

airports = pd.concat([
    airline_sample_df["origin"],
    airline_sample_df["dest"]
]).dropna().str.upper().unique()

airports_df = pd.DataFrame({"airport_code": airports})

airports_df.to_sql(
    "airports_dim",
    engine,
    if_exists="append",
    index=False
)

print("airports_dim loaded")

airports_dim loaded
